In [2]:
from bs4 import BeautifulSoup
import requests
import json
import os
import jinja2

In [4]:
#stackoverflow
def render_jinja(tpl_path, context):
    path, filename = os.path.split(tpl_path)
    return jinja2.Environment(
        loader=jinja2.FileSystemLoader(path or './')
    ).get_template(filename).render(context)

def scrape_jobs():
    url = 'https://stackoverflow.com/jobs/feed'
    data = requests.get(url)
    soup = BeautifulSoup(data.text,"lxml")
    for this_item in soup.findAll('item'):
        keys = [tag.name for tag in this_item.find_all()]
        output_dict = {}
        for key in keys:
            output_dict[key]=this_item.find(key).text
        with open('jobs/'+output_dict['guid']+'.json','w') as f:
            json.dump(output_dict,f,sort_keys = True, indent = 4,
               ensure_ascii = False)

def make_html_from_jobs():   
    for filename in os.listdir('jobs'):
        with open('jobs/'+filename,'r') as f:
            output_dict = json.load(f)
        print(len(output_dict['description']))
        context = {'jinja_dict':output_dict}
        with open('jobs_out/{}.html'.format(output_dict['guid']),'wb') as f:
            f.write(render_jinja('template.html',context).encode('utf-8'))

In [5]:
scrape_jobs()
make_html_from_jobs()

3751
3647
1831
3802
3245
2490
2420
5135
2460
2663
4450
4216
2113
2038
4334
4594
3230
911
1915
1600
5014
5154
6116
2947
2616
1987
5458
2003
3663
4147
2510
1615
3692
4635
2784
4527
2904
2905
3800
2692
4676
5588
3070
2925
3070
3020
3030
579
5775
2943
3083
3303
3398
5956
3570
2943
2922
2244
2169
3835
3054
1214
6231
2943
3070
4158
3559
3349
6574
1135
3468
2252
2196
2682
984
4840
4114
2058
2233
1903
3267
3614
1229
2336
1126
2464
3510
1371
3319
1385
2695
1636
2693
4276
562
2252
1071
935
2943
3838
3234
2708
2857
3110
2085
2099
1399
1499
2314
3560
3913
2023
6390
2634
2528
4865
3697
4165
2130
2624
1902
4771
4618
1583
1458
2142
4005
5111
2730
2355
1456
1690
2943
1284
3269
2921
4699
4133
1563
3791
4752
1932
3018
5704
2472
2322
5836
2530
3560
8043
5142
1918
2076
2894
2071
5146
3570
3702
3056
2180
3571
1718
3718
2218
7292
2621
3574
828
3663
1465
1790
3864
3302
1788
2157
5056
2943
1308
1969
1827
7764
2943
2559
3677
2904
1825
5079
4019
3092
1554
2140
2721
2360
5422
6411
8043
4896
3570
2029
3360
3355
4

In [36]:
#remote.co
import time

url = 'https://remote.co/remote-jobs/developer/'
data = requests.get(url)
soup = BeautifulSoup(data.text,"html5lib")
job_list = soup.findAll(class_='job_listing')
job_urls = []
for job in job_list:
    output_dict = {}
    url = job.find('a')['href']
    data = requests.get(url)
    soup = BeautifulSoup(data.text,"html5lib")
    job_listing = soup.find(class_='job_listing')
    output_dict['title'] = job_listing.find(class_='title_sm').text
    output_dict['category'] = 'remote.io'
    output_dict['description'] = job_listing.find(class_='job_description').text
    output_dict['url']=url
    output_dict['guid']=str(hash(url))
    with open('remote_jobs/'+output_dict['guid']+'.json','w') as f:
            json.dump(output_dict,f,sort_keys = True, indent = 4,
               ensure_ascii = False)

https://remote.co/job/software-engineer-22/ 3
https://remote.co/job/angular-front-end-developer/ 3
https://remote.co/job/senior-software-developer-6/ 3
https://remote.co/job/senior-front-end-developer-7/ 3
https://remote.co/job/lead-ux-software-engineer/ 3
https://remote.co/job/software-engineer-21/ 3
https://remote.co/job/php-magento-developer/ 3
https://remote.co/job/asp-net-developer/ 3
https://remote.co/job/operations-engineer-devops/ 3
https://remote.co/job/full-stack-lamp-js-developer/ 3
https://remote.co/job/full-stack-developer-4/ 3
https://remote.co/job/senior-web-thin-client-engineer/ 3
https://remote.co/job/firefox-telemetry-engineer/ 3
https://remote.co/job/full-stack-developer-3/ 3
https://remote.co/job/full-stack-engineer-9/ 3
https://remote.co/job/senior-software-engineer-25/ 3
https://remote.co/job/plugin-developer/ 3
https://remote.co/job/front-end-developer-16/ 3
https://remote.co/job/full-stack-engineer-8/ 3
https://remote.co/job/ab-initio-etl-developer-application-s

In [70]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier

from unidecode import unidecode
def remove_non_ascii(text):
    return unidecode(unicode(text, encoding = "utf-8"))

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42)),
 ])


Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...     penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False))])

In [83]:
categories = ['alt.atheism', 'soc.religion.christian',
               'comp.graphics', 'sci.med']
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',
    shuffle=True, random_state=42)
text_clf.fit(twenty_train.data,twenty_train.target)

job_filenames = os.listdir('remote_jobs')
test_set = []
for job_filename in job_filenames:
    with open('remote_jobs/'+job_filename,'r') as f:
            test_set.append(json.load(f)['description'])
category = text_clf.predict(test_set)
[twenty_train.target_names[category_i] for category_i in category]

['comp.windows.x',
 'sci.crypt',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'rec.sport.hockey',
 'sci.crypt',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'sci.space',
 'rec.sport.hockey',
 'rec.sport.baseball',
 'comp.windows.x',
 'comp.windows.x',
 'comp.windows.x',
 'sci.crypt',
 'soc.religion.christian',
 'misc.forsale',
 'soc.religion.christian',
 'comp.windows.x',
 'comp.windows.x',
 'sci.space',
 'talk.politics.mideast',
 'comp.windows.x',
 'misc.forsale',
 'soc.religion.christian',
 'comp.windows.x',
 'comp.windows.x',
 'soc.religion.christian']

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

'From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n'

'UX Engineer Parse.ly is a real-time content measurement layer for the entire web. Our analytics platform helps digital storytellers at some of the web’s best sites, such as Arstechnica, The Wall Street Journal, The New Yorker, TechCrunch, The Intercept, Mashable, and many more. In total, our analytics system handles over 65 billion monthly events from over 1 billion monthly unique visitors. Our entire stack is in Python and JavaScript, and our team has innovated in areas related to real-time analytics, building some of the best open source tools for working with modern stream processing technologies. Our UX/design team has also built one of the best-looking dashboards on the planet, using AngularJS and D3.js. We are now expanding our reach into the area of petabyte-scale data analysis in order to deliver insights to our customers about how news and information flows online. What is the UX team? Our User Experience (UX) team owns the core experience of Parse.ly’s customer-facing SaaS p